In [1]:
from google.cloud import storage
import os
from io import StringIO # if going with no saving csv file
import gcsfs
import dask.dataframe as dd
from dask.delayed import delayed
from pathlib import Path
import pandas as pd
import os

In [2]:
from pprint import pprint, pp
from IPython.display import display, Markdown

In [3]:
data_dir = Path(os.getenv('APP_DIR') + '/data')

userlogin_datadir = Path(data_dir, 'nanohub_metrics/userlogin')
jos_users_datadir = Path(data_dir, 'nanohub/jos_users')

In [4]:
filenames = userlogin_datadir.glob('*.parquet.gzip')
dfs = [delayed(pd.read_parquet)(fn) for fn in filenames]

dask_df = dd.from_delayed(dfs) 
dask_df = dask_df.persist()
userlogin_df = dask_df.compute(scheduler='processes')

In [5]:
display(userlogin_df.head())
display(userlogin_df.tail())

,datetime,user,uidnumber,ip,action
id,,,,,
264497646,2018-02-19 04:39:48,-,0,157.55.39.123,detect
264497647,2018-02-19 04:39:49,-,0,14.139.181.229,detect
264497648,2018-02-19 04:39:49,-,0,46.229.168.80,detect
264497649,2018-02-19 04:39:50,-,0,118.185.164.5,detect
264497650,2018-02-19 04:39:51,-,0,14.139.181.229,detect


,datetime,user,uidnumber,ip,action
id,,,,,
108497641,2015-08-03 02:14:03,-,0,68.180.229.165,detect
108497642,2015-08-03 02:14:04,-,0,66.249.69.71,detect
108497643,2015-08-03 02:14:04,-,0,68.180.229.165,detect
108497644,2015-08-03 02:14:06,-,0,74.15.149.43,detect
108497645,2015-08-03 02:14:06,-,0,61.16.171.242,detect


In [6]:
jos_users_df = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in jos_users_datadir.glob('*.parquet.gzip')
)

In [7]:
display(jos_users_df.head())
display(jos_users_df.tail())

,name,username,email,lastvisitDate,registerDate
id,,,,,
998,hubrepo hubrepo,hubrepo,nkissebe@gmail.com,NaT,2014-11-13 21:09:09
1000,CMS Admin Manager,admin,support@nanohub.org,NaT,NaT
1683,nanoHUB support,support,support@nanohub.org,2008-11-19 23:55:30,2008-11-19 22:51:04
1684,Grid Statistics,gridstat,gridstat@nanohub.org,2020-02-14 18:50:14,2008-11-18 17:29:56
1685,NCN NCN,ncn,ncn@nanohub.org,NaT,2008-11-11 19:17:04


,name,username,email,lastvisitDate,registerDate
id,,,,,
332875,Katherine Caroline,katherinecarolynd9890,katherinecarolynd9890@gmail.com,2021-07-17 09:11:38,2021-07-17 09:11:38
332876,buy IG followers from Insta4likes,insta4likes,insta4likes@protonmail.com,2021-07-17 09:42:08,2021-07-17 09:41:15
332879,Richard Boice,rboice,ricky.boice@gmail.com,2021-07-17 09:45:45,2021-07-17 09:45:45
332880,joe nick,knighten,knighten@gmail.com,2021-07-17 09:56:08,2021-07-17 09:56:08
332881,Kenneth Parker,-174570,-174570@invalid,2021-07-17 10:18:56,2021-07-17 10:18:55


In [8]:
display(userlogin_df.dtypes)

datetime     datetime64[ns]
user                 object
uidnumber             int64
ip                   object
action               object
dtype: object

In [10]:
action_types_df = userlogin_df.groupby('action')['user'].unique().reset_index()
pd.options.display.max_colwidth = 5000
display(action_types_df)
pd.reset_option('max_colwidth')

,action,user
0,astronomy,[-]
1,detect,"[-, pouyasoltan, nwdt3309, sankrish91, afreenkhursheed10, prashantt93shukla, kety, -84876, iraj.maleki.sh, bhubonphd, bhubon.ism.phd, slrommel, jclair, majid682008, dellamoris12, saavedra.pfm, ishamaini, amryoussef, iagord93, kirankabothu, surajprasad80, metallika4ever, doston6556, nitachoudhary123, misterbransito96, link, nfinan, -84749, bittu1266, raheela, mohammadm, muhammadshernasir, oves, isabelt, nlamine6, uuchecknow, -84287, jlourenco, nmnemes, devikapulse, aya.saidi, -84535, csampe, khan176, mbharathicena6, 2017rme9059, kilchytska, -84220, paula.rojas, -84878, flynn, -84834, faltens, -84879, debashree, ashraf, annajohnson, lundstro, sunny.cp.vns, halilibrahim, mastagno, monicacampos, amuthan, carmen91, bfranz, zhangsijie1995, kotszwai, bmahaja, liu1947, mmh, ameerzafar01, divyadivi1712, raousama3, nayita210, adam.kwiatkowski, jlawry, syuan.sc, tutul, li2515, amittripathi, strachan, ml2097, -72562, jgwest, atsr21, mfrr3, resmi968, esthersabestian11223, swapnalimakdey25, less76, ngbastus, biswajit025, ubhaskar, kplass, jjjarcze, mathcancer, desai61, devidassphd, r.asadpour, sydney, ...]"
2,forum,[-]
3,invalid,"[dellamoris, della89.webmaster@gmail.com, [unknown], Charlessoiva, admin, Alessia_M_28, slrommel, annmjohnson55@gmail.com, alisajones775@gmail.com, marinapal, Ykar, rpneriona, jgwest, umesha86, mainamaninfo, nathan.vanvelson@1-act.com, nkvv88, domenico.ninno@unina.it, lggslore, raghunathan@purdue.edu, somilshivhare910@gmail.com, paulmather, TimoPresidente, TimoParaPresidente, sobh1, Juan Davidsito Serrano, jd.serranom@uniandes.edu.co, DougltrApack, Donaltrtem, carlos.leon.chinchay@gmail.com, chinchay, tony.russo@vanderbilt.edu, Titan3X, s235708, qinzhao@mit.edu, zhaoqin02@gmail.com, rpw.32@live.com, imsyafa_89@yahoo.com, fuente, cp.mit777@gmail.com, cpmit777, davidhisu, lamstoresekay, Glennagofe, purplefish985@gmail.com, krishna0460@gmail.com, sumit00, sahils1696@gmail.com, akshayarora271@yahoo.com, joshimukesh282, mjakhar7665@gmail.com, dinimelayu615@gmail.com, divyanshu2308@gmail.com, a.a.koshkarev@gmail.com, carterjames5297, tonijr, adarshtiwari995@gmail.com, Lincolnhix, KennethBaf, laboratorio2018, laboratorioMateriales, kildisha, z.kudyshev@gmail.com, antonio.poilmeni@roma1.infn.it, francisco.jquintanilla@alumnos.upm.es, adervise, liuzhuo2015, lzentner, raquelhg, samava3, samava2, schowdhury, ConstanceMuscio, LeliaFap, lstirtag@gmail.com, pnayak1@asu.edu, Dennisgloni, martinschultze, kcantley, zevg@gatech.edu, charlejs, guoj, mluo12@hawk.iit.edu, ca.criollo99@uniandes.edu.co, Williamasync, wyang28, y355903682@gmail.com, zhan1193@purdue.edu, soloveron, hfennell@purdue.edu, shcha@gist.ac.kr, AeraceBoulk, mohamed.mohamed33@gmail.com, nbutt, aogilhamdean, mamifatima@gmail.com, rtrivedi, shanonstevenson, JamesFiste, hitechro122@gmail.com, ...]"
4,login,"[-84876, thomasaubert, steinrealtor, kety, dellamoris12, iraj.maleki.sh, bhubonphd, bhubon.ism.phd, slrommel, sasi, saavedra.pfm, ishamaini, amryoussef, iagord93, doston6556, misterbransito96, -84877, link, carmen91, bittu1266, suriatipaiman, imtiajkhan314, mohammadm, majid682008, muhammadshernasir, surajprasad80, olowojibola, -84806, nlamine6, faiyazmullick, jlourenco, geniusstreamapp, devikapulse, aya.saidi, alessia_m, paula.rojas, -84878, -84879, -84880, annajohnson, kirankabothu, nayita210, liu1947, mmh, fredricklancer, eduardo.saavedra.d, adam.kwiatkowski, syuan.sc, li2515, amittripathi, ml2097, leonard.ruiz00, -84881, jgwest, esthersabestian11223, sudhamalik, ngbastus, ykar, ubhaskar, kplass, desai61, devidassphd, bmirzapouebe, -84882, -84883, fuentesa, anneng, dbenham, yannikhein, rafatulfaria, yang1489, -84884, nkvv88, gmcgraw, phyzaoui, tdiclemente, christopher1993, mtorfeh, rabbani2398, emi_emi, tumamitaloka, krozman93, -84885, fabri4twork, -84886, rentalmotormurah, brendenh88, -84887, -84888, -84889, ninnodomenico, sdc22, ivanjkv, joshua, tkubis, finishmyhw, anand_raghunathan, -84890, mttrch, -84891, ...]"
5,logout,"